In [45]:
import requests
import pandas as pd
import json
import os

In [46]:
# add url
base_url = "https://api.inaturalist.org/v1"

# add iNaturalist user
user = "d_gonzalez"

In [47]:
# define request function
def get_data(endpoint, params={}):
    url = f"{base_url}/{endpoint}"
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error getting {endpoint}: {e}")
        return None

In [48]:
# store json
def to_json(data, filename):
    try:
        # # Ensure the directory exists
        # os.makedirs(os.path.join("..", "files", "raw"), exist_ok=True)
        
        # Create the full path for the file
        file_path = os.path.join("..", "files", "raw", filename)
        
        # Write the JSON data to the file
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)  # Pretty-print JSON
        print(f"Data saved to {file_path}")
    except Exception as e:
        print(f"Error saving to {file_path}: {e}")

In [49]:
# # store as json
# def to_json(data, filename):
#     try:
#         df = pd.DataFrame(data)  # Convert list of dictionaries to DataFrame
#         df.to_json(filename, index=False, encoding="utf-8")  # Save to json
#         print(f"Data saved to {filename}")
#     except Exception as e:
#         print(f"Error saving to {filename}: {e}")

In [50]:
# get observations
def get_observations(user):
    print("Getting my observations...")
    params = {"user_id": user, 
              "per_page": 30}
    data = get_data("observations", params)
    return data.get("results", []) if data else []

In [51]:
# get ids
def get_ids(user):
    print("Getting my ids...")
    params = {"user_id": user, 
              "per_page": 30}
    data = get_data("identifications", params)
    return data.get("results", []) if data else []

In [52]:
#get profile
def get_profile(user):
    print("Getting your profile...")
    params = {"q": user}
    data = get_data("users/autocomplete", params)
    return data.get("results", []) if data else []

In [53]:
# main function to bring you your data
def main():
    # Observations
    observations = get_observations(user)
    if observations:
        print(f"Retrieved {len(observations)} observations.")
        to_json(observations, "observations.json")

    # Identifications
    identifications = get_ids(user)
    if identifications:
        print(f"Retrieved {len(identifications)} identifications.")
        to_json(identifications, "identifications.json")

    # User Profile
    user_profile = get_profile(user)
    if user_profile:
        print("Retrieved user profile.")
        to_json([user_profile], "user_profile.json")

if __name__ == "__main__":
    main()

Getting my observations...
Retrieved 30 observations.
Data saved to ..\files\raw\observations.json
Getting my ids...
Retrieved 30 identifications.
Data saved to ..\files\raw\identifications.json
Getting your profile...
Retrieved user profile.
Data saved to ..\files\raw\user_profile.json


In [23]:
# Load JSON
with open("observations.json", "r", encoding="utf-8") as f: 
    observations = json.load(f)

# Convert to DataFrame and Save to CSV
df = pd.DataFrame(observations)
df.to_csv("observations_reprocessed.csv", index=False)

In [ ]:
# parse json
def parse_json():
    raw = os.path.join("..", "files", "raw")
    
    # List of JSON files in the raw folder
    json_files = [f for f in os.listdir(raw) if f.endswith('.json')]
    
    # Process each JSON file
    for json_file in json_files:
        json_path = os.path.join(raw, json_file)
        
        # Load the JSON data
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Check if the data is a list (common for iNaturalist data)
            if isinstance(data, list):
                df = pd.DataFrame(data)
            else:
                # If the JSON structure is not a simple list, adjust accordingly
                df = pd.DataFrame([data])  # Wrap in a list to handle single objects
            
            # Save the data to a CSV in the same folder
            output_csv = os.path.join(raw, json_file.replace('.json', '.csv'))
            df.to_csv(output_csv, index=False, encoding="utf-8")
            print(f"Successfully saved {json_file} as CSV.")
        
        except Exception as e:
            print(f"Error processing {json_file}: {e}")

parse_json()

Successfully saved identifications.json as CSV.
Successfully saved observations.json as CSV.
Successfully saved user_profile.json as CSV.
